# Fits files

In this notebook we show, how you can store your mock datacube in a fits file, which is the common format in which are observational data handled. We firtss create a mock IFU cube by running the RUBIX pipeline, store it then in a fits file and then lod the data from the fits file.

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
import os
from rubix.core.pipeline import RubixPipeline

# Define Illustris configuration
config_illustris = {
    "pipeline": {"name": "calc_ifu"},
    "logger": {"log_level": "DEBUG", "log_file_path": None, "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"},
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars", "gas"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        "load_galaxy_args": {"id": 11, "reuse": True},
        "subset": {"use_subset": True, "subset_size": 1000},
    },
    "simulation": {"name": "IllustrisTNG", "args": {"path": "data/galaxy-id-11.hdf5"}},
    "output_path": "output",
    "telescope": {"name": "MUSE", "psf": {"name": "gaussian", "size": 5, "sigma": 0.6}, "lsf": {"sigma": 0.5}, "noise": {"signal_to_noise": 10, "noise_distribution": "normal"}},
    "cosmology": {"name": "PLANCK15"},
    "galaxy": {"dist_z": 0.1, "rotation": {"type": "edge-on"}},
    "ssp": {"template": {"name": "BruzualCharlot2003"}},
}

# Define NIHAO configuration
config_nihao = {
    "pipeline": {"name": "calc_ifu"},
    "logger": {"log_level": "DEBUG", "log_file_path": None, "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"},
    "data": {
        "name": "NihaoHandler",
        "args": {
            "particle_type": ["stars", "gas"],
            "save_data_path": "data",
            "snapshot": "2000",
        },
        "load_galaxy_args": {"reuse": True, "id": "g8.26e11"},
        "subset": {"use_subset": True, "subset_size": 1000},
    },
    "simulation": {
        "name": "NIHAO",
        "args": {
            "path": '/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.02000',
            "halo_path": '/mnt/storage/_data/nihao/nihao_classic/g8.26e11/g8.26e11.02000.z0.000.AHF_halos',
            "halo_id": 0,
        },
    },
    "output_path": "output",
    "telescope": {"name": "MUSE", "psf": {"name": "gaussian", "size": 5, "sigma": 0.6}, "lsf": {"sigma": 0.5}, "noise": {"signal_to_noise": 1, "noise_distribution": "normal"}},
    "cosmology": {"name": "PLANCK15"},
    "galaxy": {"dist_z": 0.1, "rotation": {"type": "face-on"}},
    "ssp": {"template": {"name": "BruzualCharlot2003"}},
}

# Select which configuration to use
# Change this variable to 'illustris' or 'nihao' to switch datasets
simulation_type = "illustris"  # or "nihao"

config = config_illustris if simulation_type == "illustris" else config_nihao

# Run pipeline
pipe = RubixPipeline(config)
data = pipe.run()

In [ ]:
#NBVAL_SKIP
datacube = data.stars.datacube

img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")

# Store datacube in a fits file with header

In RUBIX we implemented a function that automaticly takes the relevant information from the config and writes it into the header. Then the header and data are stored in a fits file. All is done with the store_fits function from the rubix.core.fits module.

In [ ]:
#NBVAL_SKIP
from rubix.core.fits import store_fits

store_fits(config, data, "output/")

# Load datacube from fits file

We implemented a function to load a fits file. It is based on MPDAF, which is a package to handle MUSE IFU cubes. You can load your datacube by the following line and access all kind of information from the fitsfile.

In [ ]:
#NBVAL_SKIP
from rubix.core.fits import load_fits

cube = load_fits("output/IllustrisTNG_id11_snap99_stars_subsetTrue.fits") #if you use NIHAO, you have to insert the NIHAO fits file

In [ ]:
#NBVAL_SKIP
cube.shape

In [ ]:
#NBVAL_SKIP
cube.info()

In [ ]:
#NBVAL_SKIP
cube.primary_header

In [ ]:
#NBVAL_SKIP
import matplotlib.pyplot as plt

image1 = cube[0,:,:]

plt.figure()
image1.plot(colorbar='v', title = '$\lambda$ = %.1f (%s)' %(cube.wave.coord(1000), cube.wave.unit))
plt.show()